In [4]:
import pandas as pd
import sqlite3

#read csv file by pandas
school = pd.read_csv('ChicagoPublicSchools.csv')
school.index = school.index + 1  #start index from 1

crime = pd.read_csv('ChicagoCrimeData.csv')
crime.index = crime.index + 1

census = pd.read_csv('ChicagoCensusData.csv')
census.index = census.index + 1

#connect to database
conn = sqlite3.connect('Chicago_dataset.db')
c = conn.cursor()

#drop table if exists
c.execute(" drop table if exists CHICAGO_PUBLIC_SCHOOLS ")
c.execute(" drop table if exists CHICAGO_CRIME_DATA ")
c.execute(" drop table if exists CENSUS_DATA ")

#load datatable into database
school.to_sql('CHICAGO_PUBLIC_SCHOOLS',conn)
crime.to_sql('CHICAGO_CRIME_DATA',conn)
census.to_sql('CENSUS_DATA',conn)

In [25]:
sta = "SELECT * FROM CHICAGO_CRIME_DATA ;"
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows', 1000)
da = pd.read_sql_query(sta, conn).head()
da.style.hide_index() 

index,ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
1,3512276,HK587712,2004-08-28,047XX S KEDZIE AVE,890,THEFT,FROM BUILDING,SMALL RETAIL STORE,0,0,911,9,14.000000,58.000000,6,1155838.000000,1873050.000000,2004,41.807440,-87.703956,"(41.8074405, -87.703955849)"
2,3406613,HK456306,2004-06-26,009XX N CENTRAL PARK AVE,820,THEFT,$500 AND UNDER,OTHER,0,0,1112,11,27.000000,23.000000,6,1152206.000000,1906127.000000,2004,41.898280,-87.716406,"(41.898279962, -87.716405505)"
3,8002131,HT233595,2011-04-04,043XX S WABASH AVE,820,THEFT,$500 AND UNDER,NURSING HOME/RETIREMENT HOME,0,0,221,2,3.000000,38.000000,6,1177436.000000,1876313.000000,2011,41.815933,-87.624642,"(41.815933131, -87.624642127)"
4,7903289,HT133522,2010-12-30,083XX S KINGSTON AVE,840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,0,0,423,4,7.000000,46.000000,6,1194622.000000,1850125.000000,2010,41.743665,-87.562463,"(41.743665322, -87.562462756)"
5,10402076,HZ138551,2016-02-02,033XX W 66TH ST,820,THEFT,$500 AND UNDER,ALLEY,0,0,831,8,15.000000,66.000000,6,1155240.000000,1860661.000000,2016,41.773455,-87.706480,"(41.773455295, -87.706480471)"


In [24]:
sta = "SELECT * FROM CENSUS_DATA;"
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows', 1000)
da = pd.read_sql_query(sta, conn)
da.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78 entries, 0 to 77
Data columns (total 10 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   index                                         78 non-null     int64  
 1   COMMUNITY_AREA_NUMBER                         77 non-null     float64
 2   COMMUNITY_AREA_NAME                           78 non-null     object 
 3   PERCENT_OF_HOUSING_CROWDED                    78 non-null     float64
 4   PERCENT_HOUSEHOLDS_BELOW_POVERTY              78 non-null     float64
 5   PERCENT_AGED_16__UNEMPLOYED                   78 non-null     float64
 6   PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA  78 non-null     float64
 7   PERCENT_AGED_UNDER_18_OR_OVER_64              78 non-null     float64
 8   PER_CAPITA_INCOME                             78 non-null     int64  
 9   HARDSHIP_INDEX                                77 non-null     float

### Find the total number of crimes recorded in the CRIME table.

In [13]:
statement = "   SELECT COUNT(CASE_NUMBER) AS TOTAL_NUMBER_OF_CRIMES \
                FROM CHICAGO_CRIME_DATA"
da = pd.read_sql_query(statement, conn)
da.style.hide_index() #remove index label

TOTAL_NUMBER_OF_CRIMES
533


### List community areas with per capita income less than 11000.

In [14]:
statement = "   SELECT COMMUNITY_AREA_NAME, PER_CAPITA_INCOME \
                FROM CENSUS_DATA \
                WHERE PER_CAPITA_INCOME < 11000;"
da = pd.read_sql_query(statement, conn)
da.style.hide_index() #remove index label

COMMUNITY_AREA_NAME,PER_CAPITA_INCOME
West Garfield Park,10934
South Lawndale,10402
Fuller Park,10432
Riverdale,8201


### List all case numbers for crimes  involving minors?(children are not considered minors for the purposes of crime analysis)

In [15]:
statement = "   SELECT CASE_NUMBER, DESCRIPTION  \
                FROM CHICAGO_CRIME_DATA \
                WHERE DESCRIPTION LIKE '%MINOR%'"
da = pd.read_sql_query(statement, conn)
da.style.hide_index() #remove index label

CASE_NUMBER,DESCRIPTION
HL266884,SELL/GIVE/DEL LIQUOR TO MINOR
HK238408,ILLEGAL CONSUMPTION BY MINOR


### List all kidnapping crimes involving a child?

In [16]:
statement = "   SELECT CASE_NUMBER, PRIMARY_TYPE, DESCRIPTION  \
                FROM CHICAGO_CRIME_DATA \
                GROUP BY PRIMARY_TYPE \
                HAVING PRIMARY_TYPE like 'KIDNAPPING' AND DESCRIPTION LIKE '%CHILD%' "
da = pd.read_sql_query(statement, conn)
da.style.hide_index() #remove index label

CASE_NUMBER,PRIMARY_TYPE,DESCRIPTION
HN144152,KIDNAPPING,CHILD ABDUCTION/STRANGER


### What kinds of crimes were recorded at schools?

In [17]:
statement = "   SELECT distinct(PRIMARY_TYPE) \
                FROM CHICAGO_CRIME_DATA \
                WHERE LOCATION_DESCRIPTION LIKE 'SCHOOL%' "
#statement = " select distinct(LOCATION_DESCRIPTION) FROM CHICAGO_CRIME_DATA" 
da = pd.read_sql_query(statement, conn)
da.style.hide_index() #remove index label

PRIMARY_TYPE
BATTERY
CRIMINAL DAMAGE
NARCOTICS
ASSAULT
CRIMINAL TRESPASS
PUBLIC PEACE VIOLATION


### List the average safety score for each type of school.

In [18]:
statement =     'SELECT "Elementary, Middle, or High School", AVG(SAFETY_SCORE) AS AVERAGE_SAFETY_SCORE \
                 FROM CHICAGO_PUBLIC_SCHOOLS \
                 GROUP BY "Elementary, Middle, or High School";'
da = pd.read_sql_query(statement, conn)
da.style.hide_index() #remove index label

"Elementary, Middle, or High School",AVERAGE_SAFETY_SCORE
ES,49.520384
HS,49.623529
MS,48.000000


### List 5 community areas with highest % of households below poverty line

In [19]:
statement = "   SELECT COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY \
                FROM CENSUS_DATA\
                ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC\
                LIMIT 5"
da = pd.read_sql_query(statement, conn)
da.style.hide_index() #remove index label

COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
Riverdale,56.500000
Fuller Park,51.200000
Englewood,46.600000
North Lawndale,43.100000
East Garfield Park,42.400000


### Which community area is most crime prone?

In [20]:
statement = "   SELECT COMMUNITY_AREA_NUMBER, COUNT(COMMUNITY_AREA_NUMBER) AS NUMBER_OF_CRIME \
                FROM CHICAGO_CRIME_DATA \
                GROUP BY COMMUNITY_AREA_NUMBER \
                ORDER BY 2 DESC \
                LIMIT 1"
da = pd.read_sql_query(statement, conn)
da.style.hide_index() #remove index label

COMMUNITY_AREA_NUMBER,NUMBER_OF_CRIME
25.000000,43


### Use a sub-query to find the name of the community area with highest hardship index

In [21]:
statement = "   SELECT COMMUNITY_AREA_NUMBER, COMMUNITY_AREA_NAME, HARDSHIP_INDEX \
                FROM CENSUS_DATA \
                WHERE HARDSHIP_INDEX = (SELECT MAX(HARDSHIP_INDEX) FROM CENSUS_DATA) "
da = pd.read_sql_query(statement, conn)
da.style.hide_index() #remove index label

COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,HARDSHIP_INDEX
54.000000,Riverdale,98.000000


### Use a sub-query to determine the Community Area Name with most number of crimes?

In [22]:
statement = "   SELECT COMMUNITY_AREA_NAME, MAX(CRIME) AS MOST_NUMBER_OF_CRIMES \
                FROM \
                    (SELECT cd.COMMUNITY_AREA_NAME, count(ccd.COMMUNITY_AREA_NUMBER) as CRIME \
                    FROM CHICAGO_CRIME_DATA ccd,CENSUS_DATA cd \
                    WHERE ccd.COMMUNITY_AREA_NUMBER = cd.COMMUNITY_AREA_NUMBER \
                    GROUP BY cd.COMMUNITY_AREA_NAME) "
da = pd.read_sql_query(statement, conn)
da.style.hide_index() #remove index label

COMMUNITY_AREA_NAME,MOST_NUMBER_OF_CRIMES
Austin,43
